## Chapter 12- correcting errors
### Introduction
The correcting process will vary, depending on your application. 
- For example, you may have a large data warehouse where you can 
    - delete observations with errors (or possible errors) or 
    - replace possible data errors with missing values or with the correct values whenever possible. 
- For other applications, you may need to return to the original data forms, determine the correct values, and replace incorrect data values with corrected values.

### 1. Hard coding corrections
For small data sets with only a few corrections to be made, you might consider hard coding your corrections.

__Program 12.1: Hard Coding Corrections Using a DATA Step__


In [1]:
libname Clean '/folders/myfolders/ban110'; 
proc print data=Clean.patients; 
run;

SAS Connection established. Subprocess id is 3440



Obs,PATNO,GENDER,VISIT,HR,SBP,DBP,DX,AE
1,,M,11/11/1998,90,190,100,,0
2,001,M,11/11/1998,88,140,80,1,0
3,002,F,11/13/1998,84,120,78,X,0
4,002,F,11/13/1998,84,120,78,X,0
5,003,X,10/21/1998,68,190,100,3,1
6,003,M,11/12/1999,58,112,74,,0
7,004,F,01/01/1999,101,200,120,5,A
8,006,,06/15/1999,72,102,68,6,1
9,006,F,07/07/1999,82,148,84,1,0
10,007,M,.,88,148,102,,0


In [2]:
*This program corrects errors in the Patients data set;


data clean.patients_Chap12; 
    set clean.patients; 
    * file print; 
    array Char_Vars[3] Patno Gender Dx ; 
    do i =1 to 3; 
        Char_vars[i]= upcase (Char_vars[i]);
    end; 
    
    if patno='003' then SBP=110; 
    if patno='011' then Dx='530'; 
    if Patno='023' then do;
        SBP=146;
        DBP=98; 
        end; 
    drop i; 
run; 


In [52]:
data _null_;
    set Clean.patients_Chap12;
    if patno in ('003','011','023') then put patno= ;
run;

#### <font color=magenta> Q1. Activity </font> 
Write a code to delete the observations where patno is missing and run the code 

In [5]:
data Clean.patients_removeMissing;
    set Clean.patients_Chap12;
    if missing(patno) then delete;
run;

proc print data= Clean.patients_removeMissing;
run;



Obs,PATNO,GENDER,VISIT,HR,SBP,DBP,DX,AE
1,001,M,11/11/1998,88,140,80,1,0
2,002,F,11/13/1998,84,120,78,X,0
3,002,F,11/13/1998,84,120,78,X,0
4,003,X,10/21/1998,68,110,100,3,1
5,003,M,11/12/1999,58,110,74,,0
6,004,F,01/01/1999,101,200,120,5,A
7,006,,06/15/1999,72,102,68,6,1
8,006,F,07/07/1999,82,148,84,1,0
9,007,M,.,88,148,102,,0
10,008,F,08/08/1998,210,.,.,7,0


#### Comments: 
- you name the new data set to include a revision date, so that you can keep track of your data sets if you make more corrections in the future. 
- Also, you will want to save this program with a name such as Update_01Jan2017.sas so that you can re-create the corrected data set from the original data should that need arise. 
- Never overwrite your original data set. It is also strongly recommended that you insert a comment to document the source of your data changes.
- In addition to correcting several values, this program also converts each of the four character variables to uppercase.

### 2. Describing Named Input
You are most likely familiar with three ways of reading raw data in a SAS data set: 
- list input (used for delimited data), 
- column input (where you specify the starting and ending columns for each variable), and 
- formatted input (where you use pointers and informats). 

There is one other method for reading raw data called named input.

__Program 12.2: Describing Named Input__

In [21]:
data Named; 
    length Char $3; 
    informat Date mmddyy10.; 
    input x=
          y=
          char=
          date= ; 
datalines;
x=3 y=4 Char=abc Date=10/21/2010 
y=7
date=11/12/2016 Char=xyz x=9
;

proc print data=Named; 
run;





Obs,Char,Date,x,y
1,abc,18556,3,4
2,,.,.,7
3,xyz,20770,9,.


#### comments

- To use named input, you include an equal sign after each variable name in the INPUT statement. 
- This input method also requires you to include a variable name, an equal sign, and the value to be read in the raw data file.

- In order to tell SAS that Char is a character variable with length 3 and that Date needs an MMDDYY10. informat, you use a LENGTH statement for the variable Char and an INFORMAT statement for the variable Date. (Note: You could have used the INFORMAT statement and assigned an informat of $3. for Char instead of using a LENGTH statement.)

- You can enter your data in any order and if you leave out variables, they will automatically be given missing values.

__Program 12.3: Using Named Input to Make Corrections__

In [43]:
data patients_corrections_11Feb2019; 
    length Patno $3
            Gender $1
            dx $3
            ae $1; 
    informat Visit mmddyy10.;
    format visit date9.; 
    input patno=
          Gender=
          Visit=
          HR=
          SBP=
          DBP=
          Dx=
          AE= ; 
          
datalines; 
Patno=003 SBP=110 
Patno=023 SBP=146 DBP=98
Patno=027 Gender=F
Patno=045 HR=90
; 

proc print data=patients_corrections_11Feb2019; 
run;


Obs,Patno,Gender,dx,ae,Visit,HR,SBP,DBP
1,003,,,,.,.,110,.
2,023,,,,.,.,146,98
3,027,F,,,.,.,.,.
4,045,,,,.,90,.,.


#### comments: 
- You use a LENGTH statement for the variables Patno, Account_No, Dx, and Gender. This sets the appropriate lengths for these variables and declares them to be character. 
- An INFORMAT statement indicates that the visit date is to be read with the MMDDYY10. informat. Remember, you still need to format the variable Visit. Here the Date9 format is used.

The question is, what do you do with this data set? Let's take a moment to review the UPDATE statement.

### 3. Reviewing the UPDATE Statement
- If you MERGE two data sets that share variable names, the value from the right-most data set will replace the value from the data set to its left, even if the value in the right-most data set is a missing value. 
- However, if you use an UPDATE statement instead of a MERGE statement, a missing value for a variable in the right-most data set will not replace the value in the data set to its left. 
- The right-most data set is sometimes called the transaction data set, and it is used to update values in the master data set.

A demonstration of the how the UPDATE statement works is shown below with  two small data sets, INVENTORY and TRANSACTION.

__Program 9-4: Demonstrating How UPDATE Works__

In [26]:
data inventory;
    length PartNo $ 3;
    input PartNo $ Quantity Price;
datalines;
133 200 10.99
198 105 30.00
933 45 59.95
;

data transaction;
    length PartNo $ 3;
    input Partno=
         Quantity=
          Price=;

datalines;
PartNo=133 Quantity=195
PartNo=933 Quantity=40 Price=69.95
;

proc sort data=inventory;
by PartNo;
run; 

proc sort data=transaction;
by PartNo;
run; 


data inventory_11Feb2019;
    update inventory transaction;
    by partno; 
run; 

proc print data=inventory; 
run;

proc print data=transaction; 
run;

proc print data=inventory_11Feb2019; 
run;


Obs,PartNo,Quantity,Price
1,133,200,10.99
2,198,105,30.00
3,933,45,59.95
Obs,PartNo,Quantity,Price
1,133,195,.
2,933,40,69.95
Obs,PartNo,Quantity,Price
1,133,195,10.99
2,198,105,30.00
3,933,40,69.95


#### comments: 
-The TRANSACTION data set only contains values for two of the three part numbers. You have an updated quantity for part number 133, and for part number 933, you have a new Quantity and Price


#### <font color=magenta> Q2. Activity</font> 
reuse the update function and apply the corrections defined in patients_corrections_11Feb2019 on the original patients dataset Clean.patients. Give the updated patients dataset a new name (patients_ch12Q2), and print it. 



In [47]:
proc sort data = Clean.patients;
    by patno;
run;

proc sort data = patients_corrections_11Feb2019;
    by patno;
run;

data patients_ch12Q2;
    update Clean.patients patients_corrections_11Feb2019;
    by patno; 
run; 

title "updated data in patients dataset";
proc print data = patients_ch12Q2;
run;

Obs,PATNO,GENDER,VISIT,HR,SBP,DBP,DX,AE
1,,M,11/11/1998,90,190,100,.,0
2,001,M,11/11/1998,88,140,80,1,0
3,002,F,11/13/1998,84,120,78,.,0
4,002,F,11/13/1998,84,120,78,.,0
5,003,X,10/21/1998,68,110,100,3,1
6,003,M,11/12/1999,58,112,74,.,0
7,004,F,01/01/1999,101,200,120,5,.
8,006,,06/15/1999,72,102,68,6,1
9,006,F,07/07/1999,82,148,84,1,0
10,007,M,.,88,148,102,.,0


###  4. Replacing Missing Values 

Replacing missing values can range from a simple replacement with 0 or another predefined constant to a complex calculation of replacement values. We will see how PROC STANDARD can be used to replace missing values.

#### 4.1. Using PROC STANDARD for Replacement: Imputation using the mean

The following example shows how the REPLACE statement in PROC STANDARD can be used to replace missing values by the mean of the non-missing values. 

In the following results, we see that the missing values have been replaced by their mean 38:

In [1]:
DATA TEST; 
    INPUT AGE @@;
    CARDS;
    12 60 . 24 . 50 48 34 .
    ;
 RUN;

proc print data=TEST; 
run; 



SAS Connection established. Subprocess id is 11614



Obs,AGE
1,12
2,60
3,.
4,24
5,.
6,50
7,48
8,34
9,.


In [3]:
proc standard data=test replace print out=test_imputed;
    var age;
run; 

proc print data=test_imputed;
run;



#### <font color=magenta> Q3. Activity </font> 
Use patients dataset, replace all missing numerical values for the variable HR with the mean.   

In [49]:
title "Impute data in HR column in patients dataset";
proc standard data=clean.patients replace print out=clean.patients_Q3;
    var hr;
run;

title "New dataset with missing HR value imputed";
proc print data = clean.patients_Q3;
run;


#### 4.2. Standarization: imputation using z score

In [2]:
proc standard data=test replace mean=0 std=1 print out=test_zscore;
    var age;
run; 

proc print data=test_zscore;
run;

### 5. Searching for specific numeric value 


- You start out by creating a character format (it was called $Count_Missing in this example) that has only two value ranges, one for missing values and the other for everything else. Using this format, you can have PROC FREQ count missing and non-missing values for you.

- Notice also that it is necessary to use the SAS keyword _CHARACTER_ in the TABLES statement (or to provide a list of character variables). 

- PROC FREQ can produce frequency tables for numeric as well as character variables. Finally, the TABLES option MISSING is an instruction to include the missing values in the body of the frequency listing.
	
    Note:  	

 If you use the MISSING option with PROC FREQ and you request percentages, the percentage calculation uses all the values, missing and non-missing, in the denominator rather than just the number of non-missing values.
 
 Specific values such as 999 or 9999 are sometimes used to denote missing values. 

In order to demonstrate how to locate these special values, run Program 7.7 (below) to create a data set (called Test) that contains some numeric and character variables. 

A number of the numeric variables have values of 999. This practice is quite popular with people who use SPSS (Statistical Package for the Social Sciences) where a statement "Assign Missing 999" automatically replaces all numeric values of 999 to a missing value. 

If you are given a data set that contains special values (such as 999) to represent missing values, you might want to investigate which variables have this value and how many times these values appear.

__Program 7.7: Creating a Test Data Set__

In [31]:
***Create test data set;
data Test;
   input X Y A $ X1-X3 Z $;
datalines;
1 2 X 3 4 5 Y
2 999 Y 999 1 999 J
999 999 R 999 999 999 X
1 2 3 4 5 6 7
;

proc print data=test; 
run;

Obs,X,Y,A,X1,X2,X3,Z
1,1,2,X,3,4,5,Y
2,2,999,Y,999,1,999,J
3,999,999,R,999,999,999,X
4,1,2,3,4,5,6,7


Program 7.8 searches a SAS data set for all numeric variables set to a specific value and produces a report, which shows the variable name and the observation where the specific value was found. 

The trick in this program is the VNAME function. This function returns the variable name corresponding to an array element. Program 7.8 searches a SAS data set for a specific value. 

__Program 7.8: Detecting and Counting Special Values (999 in this Example)__

In [34]:
***Program to detect the specified values;
title "Looking for Values of 999 in Data Set Test";
data _null_;
   set Test;
   file print;
   array Nums[*] _numeric_;
   length Varname $ 32;
   do iii = 1 to dim(Nums);
      if Nums[iii] = 999 then do;
         Varname = vname(Nums[iii]);
         put "Value of 999 found for variable " Varname
             "in observation " _n_;
      end;
   end;
   drop iii;
run;

- One key to this program is the use of _NUMERIC_ in the ARRAY statement.  Because this ARRAY statement follows the SET statement, the array Nums will contain all the numeric variables in data set Test. 
- The next step is to examine each of the elements in the Nums array, determine if a value of 999 is found, and then determine the variable name associated with that array element. 
- The DO loop uses the index variable iii in the hopes that there will not be any variables in the data set to be tested with that name. If there were, the DO loop counter would replace the value in the data set being tested.

- Now for the trick! As you search for values of 999 for each of the numeric variables, you can use the VNAME function to return the variable name that corresponds to the array element. In this program, the variable name is stored in the variable Varname. All that is left to do is to write out the variable names and observation numbers. 

### 6. Converting Values Such as 999 to a SAS Missing Value
 
 Although SAS does not have a statement such as "Assign Missing 999," it is quite easy to convert any special value to a SAS missing value for all numeric variables in a data set. As an example, the program below converts all values of 999 in data set Test (used in the previous section) to a SAS missing value:
 
 __Program 7.10: Converting All Values of 999 to a SAS Missing Value__

In [4]:
data Set_999_to_Missing;
   set Test;
   array Nums[*] _numeric_;
   do iii = 1 to dim(Nums);
      if Nums[iii] = 999 then Nums[iii] = .;
   end;
   drop iii;
run;

proc print data=Set_999_to_Missing; 
run;

Obs,X,Y,A,X1,X2,X3,Z
1,1,2,X,3,4,5,Y
2,2,.,Y,.,1,.,J
3,.,.,R,.,.,.,X
4,1,2,3,4,5,6,7


 This program is very similar to Program 7.8, except this time when a value of 999 is found, a missing value is substituted